In [1]:
%matplotlib nbagg

In [2]:
import bempp.api as bpp
import bempp.core as core
import bempp.api as api
import numpy as np
from bempp.core.grid import grid_from_sphere
# from bempp.operators.boundary import sparse
# from bempp.operators.boundary import laplace as boundary_laplace
# from bempp.operators.potential import laplace as potential_laplace

INFO:BEMPP:Dolfin could not be imported. FEM/BEM coupling with FEniCS not available.


In [3]:
import bempp

In [4]:
gridfile = bpp.file_interfaces.FileReader(file_name="../../meshes/sphere-h-0.1.msh")
grid = gridfile.grid

In [5]:
try:
    from ipyparallel import Client
except ImportError as exc:
    from IPython.parallel import Client

In [6]:
rc = Client()
rc.ids
dview = rc[:] # use all engines

In [7]:
with dview.sync_imports():
    import bempp
    import bempp.api
    import bempp.core
    import numpy

importing bempp on engine(s)
importing bempp.api on engine(s)
importing bempp.core on engine(s)
importing numpy on engine(s)


In [8]:
elements = grid.leaf_view.elements
vertices = grid.leaf_view.vertices
domain_indecies = grid.leaf_view.domain_indices
dview.push((dict(elements=elements, vertices=vertices, domain_indecies=domain_indecies)))

<AsyncResult: finished>

In [9]:
%%px --local
newgrid = bempp.api.grid_from_element_data(vertices, elements, domain_indecies)
piecewise_const_space = bempp.api.function_space(newgrid,"DP",0,)
piecewise_lin_space = bempp.api.function_space(newgrid,"P",1)

In [10]:
%%px --local 
def dirichlet_data(x,n,domain_index,result):
    result[0] = numpy.log(((x[0]-.9)**2+x[1]**2+x[2]**2)**(0.5))

In [11]:
%%px --local
from bempp.api import GridFunction
dirichlet_fun = GridFunction(piecewise_lin_space, dual_space=piecewise_const_space, fun=dirichlet_data)

INFO:BEMPP:Operator: IDENTITY. START ASSEMBLY. Dimensions: (1287,2570). Assembly Type: sparse
INFO:BEMPP:Operator: IDENTITY. FINISHED ASSEMBLY. Time: 0.01835489273071289 seconds


In [12]:
%%px --local
idp = bempp.api.operators.boundary.sparse.identity(piecewise_lin_space, piecewise_lin_space, piecewise_const_space)
dlp = bempp.api.operators.boundary.laplace.double_layer(piecewise_lin_space, piecewise_lin_space, piecewise_const_space)
slp = bempp.api.operators.boundary.laplace.single_layer(piecewise_const_space, piecewise_lin_space, piecewise_const_space)

In [13]:
bempp.api.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
operator = slp
result = bempp.api.assemble_dense_block(operator, bempp.api.ALL, bempp.api.ALL, operator.domain, operator.dual_to_range)
actual = bempp.api.as_matrix(result)
expected = bempp.api.as_matrix(operator.weak_form())
numpy.testing.assert_allclose(actual, expected)

INFO:BEMPP:Operator: SLP. START ASSEMBLY. Dimensions: (2570,2570). Assembly Type: dense
INFO:BEMPP:Operator: SLP. FINISHED ASSEMBLY. Time: 1.141158103942871 seconds


In [14]:
from bempp.api.utils.parallel import calculateblocks,gatherresults

In [15]:
@dview.parallel(block=True)
def my_run_in_parallel(limits):
    global actual
    bempp.api.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
    actual = bempp.api.as_matrix(bempp.api.assemble_dense_block(operator,
                                                                limits[0],
                                                                limits[1],
                                                                operator.domain,
                                                                operator.dual_to_range))

In [16]:
def run_single(operator):
    result = bempp.api.assemble_dense_block(operator,
                                            bempp.api.ALL,
                                            bempp.api.ALL,
                                            operator.domain,
                                            operator.dual_to_range)
    actual = bempp.api.as_matrix(result)
    expected = bempp.api.as_matrix(operator.weak_form())
    return expected

In [17]:
%%px --local
operator = dlp

In [18]:
chunks, nrows, ncols = calculateblocks(operator.dual_to_range.global_dof_count,
                                       operator.domain.global_dof_count,
                                       len(dview))
my_run_in_parallel.map(chunks)
gatheredresults = gatherresults(dview, 'actual', nrows, ncols)
expected = run_single(operator)

expected2 = bempp.api.as_matrix(operator.weak_form())
numpy.testing.assert_allclose(expected, expected2)
numpy.testing.assert_allclose(gatheredresults, expected)

INFO:BEMPP:Operator: DLP. START ASSEMBLY. Dimensions: (1287,2570). Assembly Type: dense
INFO:BEMPP:Operator: DLP. FINISHED ASSEMBLY. Time: 1.5182311534881592 seconds
